In [1]:
from pyogrio import read_dataframe,write_dataframe
import geopandas as gpd
import os,glob,sys,time,re
import pandas as pd
import numpy as np
import multiprocessing as mp
from parallel_pandas import ParallelPandas
from tqdm.auto import tqdm
ParallelPandas.initialize(n_cpu=24, split_factor=24)

### get basin boundary files

In [25]:
basins = glob.glob('../../data/GRIT/full_catchment/GRIT_full_catchment_*_EPSG8857_simplify_final.gpkg')
if not os.path.exists('../basin_boundary'):
    os.mkdir('../basin_boundary')
for basin in basins:
    gdf = read_dataframe(basin)
    
    # difference between ohdb_darea and grit_darea less than 20%
    gdf['bias'] = np.abs(gdf.grit_darea - gdf.ohdb_darea_hydrosheds) / gdf.ohdb_darea_hydrosheds * 100
    gdf1 = gdf.loc[gdf.bias<=20,:]
    
    # darea greater than 125 km2 to ensure at least one grid cell
    gdf1 = gdf1.loc[gdf1.grit_darea>=125,:]

    gdf1['segment_id'] = gdf1.segment_id.astype(int).astype(str)
    gdf1['reach_id'] = gdf1.segment_id.astype(int).astype(str)
    gdf1 = gdf1.rename(columns={'grit_darea':'gritDarea','ohdb_darea_hydrosheds':'ohdbDarea1','ohdb_darea':'ohdbDarea0'})
    
    # save
    basin1 = os.path.basename(basin)
    write_dataframe(gdf1, f'../basin_boundary/{basin1[:-5]}'+'_125km2.gpkg')
    gdf1 = gdf1.to_crs('epsg:4326')
    basin1 = re.sub('EPSG8857','EPSG4326',basin1)
    write_dataframe(gdf1, f'../basin_boundary/{basin1[:-5]}'+'_125km2.shp')
    print(basin)

../../data/GRIT/full_catchment/GRIT_full_catchment_AS_EPSG8857_simplify_final.gpkg
../../data/GRIT/full_catchment/GRIT_full_catchment_AF_EPSG8857_simplify_final.gpkg
../../data/GRIT/full_catchment/GRIT_full_catchment_SA_EPSG8857_simplify_final.gpkg
../../data/GRIT/full_catchment/GRIT_full_catchment_EU_EPSG8857_simplify_final.gpkg
../../data/GRIT/full_catchment/GRIT_full_catchment_NA_EPSG8857_simplify_final.gpkg
../../data/GRIT/full_catchment/GRIT_full_catchment_SI_EPSG8857_simplify_final.gpkg
../../data/GRIT/full_catchment/GRIT_full_catchment_SP_EPSG8857_simplify_final.gpkg


### select OHDB gauge and calculate streamflow indices

In [3]:
def cleanQ(df):
    # eliminate invalid records
    df1 = df.loc[df.Q.apply(lambda x: not isinstance(x, str)),:]
    df2 = df.loc[df.Q.apply(lambda x: isinstance(x, str)),:]
    try:
        df2 = df2.loc[df2.Q.str.match('\d+'),:]
    except:
        pass
    df = pd.concat([df1, df2])
    df['Q'] = df.Q.astype(np.float32)
    return df

def del_unreliableQ(df):
    '''observations less than 0 were flagged as
        suspected, and (b) observations with more than ten consecutive
        equal values greater than 0 were flagged as suspected'''
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').set_index('date')
    index = pd.date_range(df.index[0], df.index[-1], freq = 'D')
    df = df.reindex(index).fillna(0)
    df1 = df.diff()
    df1 = df1.where(df1==0, 1).diff()
    start = np.where(df1.values==-1)[0]
    end = np.where(df1.values==1)[0]
    if len(start) == 0 or len(end) == 0:
        # must no less than zero
        df = df.loc[df.Q>=0,:]
        return (df)
    if start[0] > end[0]:
        start = np.array([0]+start.tolist())
    if start[-1] > end[-1]:
        end = np.array(end.tolist()+[df1.shape[0]+10])
    duration = end - start
    start = start[duration>=10]
    end = end[duration>=10]
    del_idx = np.array([item for a,b in zip(start,end) for item in np.arange(a+1,b+2).tolist()])
    del_idx = del_idx[del_idx<df.shape[0]]
    if len(del_idx) > 0:
        df.drop(df.index[del_idx], inplace = True)
    # must no less than zero
    df = df.loc[df.Q>=0,:]
    return (df)

def main(par, scale = 'season'):
    ohdb_id, Darea = par
    df = pd.read_csv(os.environ['DATA']+f'/data/OHDB/OHDB_v0.2.3/OHDB_data/discharge/daily/{ohdb_id}.csv')
    # read
    df = cleanQ(df)
    # quality check
    df = del_unreliableQ(df)
    # only retain records with at least 328 observations (90%) are required
    tmp = df.resample('Y')['Q'].agg(countDay = lambda x:x.shape[0])
    if tmp.loc[tmp.countDay>=328,:].shape[0] == 0:
        return
    years = tmp.loc[(tmp.countDay>=328)&(tmp.index.year>=1982),:].index.year.tolist()
    if tmp.loc[(tmp.countDay>=300)&(tmp.index.year==2023),:].shape[0] > 0:
        years = years + [2023]
    df = df.loc[df.index.year.isin(years),:]
    # only retain gauge with at least 20 years of AMS during 1982-2023
    if len(years) < 20:
        return
    # reindex
    newindex = pd.date_range(df.index.values[0], df.index.values[-1], freq = 'D')
    df = df.reindex(newindex)
    # 7-day moving average
    df = df.rolling(7).mean().dropna()
    df['year'] = df.index.year
    df['season'] = 'DJF'
    df.loc[(df.index.month>=3)&(df.index.month<=5),'season'] = 'MAM'
    df.loc[(df.index.month>=6)&(df.index.month<=8),'season'] = 'JJA'
    df.loc[(df.index.month>=9)&(df.index.month<=11),'season'] = 'SON'
    if scale == 'year':
        # count observations and calculate Qmax7 and Qmin7 for each year
        df1 = df.groupby('year')['Q'].agg(countDay = lambda x:x.shape[0], 
                                        Qmax7 = lambda x:x.max(),
                                        Qmin7 = lambda x:x.min(),
                                        Qmax7date = lambda x:x.idxmax(),
                                        Qmin7date = lambda x:x.idxmin(),
                                        )
    elif scale == 'season':
        # count observations and calculate Qmax7 and Qmin7 for each season
        df1 = df.groupby([scale,'year'])['Q'].agg(countDay = lambda x:x.shape[0], 
                                        Qmax7 = lambda x:x.max(),
                                        Qmin7 = lambda x:x.min(),
                                        Qmax7date = lambda x:x.idxmax(),
                                        Qmin7date = lambda x:x.idxmin(),
                                        )
        df1 = df1.loc[df1.countDay>=90,:] # at least 90 days of records to calculate seasonal extremes
    else:
        raise Exception('scale must be season or year')
    df1['Qmax7date'] = pd.to_datetime(df1['Qmax7date'])
    df1['Qmin7date'] = pd.to_datetime(df1['Qmin7date'])
    
    # if scale == 'season':
    #     # keep events independent
    #     thres = 5 + np.log(Darea * 0.386102) # thres for Qmax7
    #     df1_Qmax = df1[['Qmax7','Qmax7date']].sort_values('Qmax7date')
    #     df1_Qmax = df1_Qmax.loc[~(df1_Qmax.Qmax7date.diff().dt.days<thres),:]
    #     thres = 30 # thres for Qmin7
    #     df1_Qmin = df1[['Qmin7','Qmin7date']].sort_values('Qmin7date')
    #     df1_Qmin = df1_Qmin.loc[~(df1_Qmin.Qmin7date.diff().dt.days<thres),:]
    #     df1 = pd.concat([df1_Qmax, df1_Qmin], axis = 1)

    # Qmax7 and Qmin7 must not be lower than zero
    df1['Qmax7'] = df1.Qmax7.where(df1.Qmax7>=0, np.nan)
    df1['Qmin7'] = df1.Qmin7.where(df1.Qmin7>=0, np.nan)

    # # Qmax7 cannot be lower than 50% percentile of daily discharges between 1982-2023
    # q = df.loc[df.Q>0,'Q'].quantile(0.5)
    # df1['Qmax7'] = df1.Qmax7.where(df1.Qmax7 >= q, np.nan)
    # # Qmin7 cannot be greater than 50% percentile of daily discharges between 1982-2023
    # df1['Qmin7'] = df1.Qmin7.where(df1.Qmin7 <= q, np.nan)

    if df1.shape[0] == 0:
        return
    df1 = df1.reset_index()
    df1['ohdb_id'] = ohdb_id
    print(ohdb_id)
    return (df1)

if __name__ == '__main__':
    # if not os.path.exists('../data/OHDB_metadata_subset.csv'):
    #     # select gauges that have good basin boundary
    #     df = pd.read_csv('../../data/OHDB/OHDB_v0.2.3/OHDB_metadata/OHDB_metadata.csv')
    #     df1 = []
    #     for fname in glob.glob('../basin_boundary/GRIT*8857*'):
    #         gdf = read_dataframe(fname, read_geometry = False)
    #         print(gdf.shape, gdf.ohdb_id.unique().shape)
    #         df1.append(df.loc[df.ohdb_id.isin(gdf.ohdb_id.unique()),:])
    #     df1 = pd.concat(df1)
    #     df1.to_csv('../OHDB_metadata_subset.csv', index = False)
    # else:
    df1 = pd.read_csv('../data/basin_attributes.csv')
    print(df1.shape)
    ohdb_ids = df1.ohdb_id.values
    pool = mp.Pool(48)
    pars = df1[['ohdb_id','gritDarea']].values.tolist()
    df = pool.map(main, pars)
    df = pd.concat(df)
    df.to_csv('../data/dis_OHDB_seasonal4_Qmin7_Qmax7_1982-2023.csv', index = False)
    print(df.Qmin7.isna().sum(), df.Qmax7.isna().sum())

(10717, 63)
OHDB_001000705OHDB_014018150OHDB_014019391OHDB_014011858OHDB_014030677OHDB_014020473

OHDB_008004429OHDB_007009037

OHDB_007005041
OHDB_003000053OHDB_006002577OHDB_011000604OHDB_012001995OHDB_006004656

OHDB_014010699


OHDB_014023472
OHDB_011000924OHDB_014011019

OHDB_006003449OHDB_014031282OHDB_010000050OHDB_012000686OHDB_014030718OHDB_011000556

OHDB_009000845OHDB_002000683
OHDB_007009994OHDB_014039706OHDB_006004441
OHDB_012002737OHDB_014021493OHDB_007008845OHDB_009000271
OHDB_011001039


OHDB_007009859OHDB_014003918OHDB_006002623OHDB_007009733OHDB_008000953OHDB_014000054OHDB_011001343











OHDB_011000701
OHDB_008001574


OHDB_011001432





OHDB_007008819
OHDB_011000484
OHDB_007004811
OHDB_007008093

OHDB_014020484
OHDB_014022562OHDB_011001015OHDB_006001966OHDB_014012189OHDB_007006237
OHDB_002000684OHDB_014033099OHDB_011000415OHDB_014021442OHDB_009000844
OHDB_006003693OHDB_014022595OHDB_011000217OHDB_006002925OHDB_007009635OHDB_011000022
OHDB_003000163

OHDB_00700

### subset basin boundary files again

In [10]:
df = pd.read_csv('../dis_OHDB_Qmin7_Qmax7_1982-2023.csv')
ohdb_ids = df.ohdb_id.unique()
for fname in glob.glob('../basin_boundary/GRIT*8857*'):
    gdf = read_dataframe(fname)
    gdf = gdf.loc[gdf.ohdb_id.isin(ohdb_ids),:]
    write_dataframe(gdf, fname[:-5]+'_subset.gpkg')
    print(fname)

../basin_boundary/GRIT_full_catchment_AS_EPSG8857_simplify_final_125km2.gpkg
../basin_boundary/GRIT_full_catchment_AF_EPSG8857_simplify_final_125km2.gpkg
../basin_boundary/GRIT_full_catchment_SA_EPSG8857_simplify_final_125km2.gpkg
../basin_boundary/GRIT_full_catchment_EU_EPSG8857_simplify_final_125km2.gpkg
../basin_boundary/GRIT_full_catchment_NA_EPSG8857_simplify_final_125km2.gpkg
../basin_boundary/GRIT_full_catchment_SI_EPSG8857_simplify_final_125km2.gpkg
../basin_boundary/GRIT_full_catchment_SP_EPSG8857_simplify_final_125km2.gpkg


### use catch_mean_GLHYMPS_GLiM.py to get catchment average subsurface characteristics

### calculate number of upstream dams

In [ ]:
gdf_dam = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_dams.shp')
gdf_dam = gdf_dam.to_crs('espg:8857')
gdf = read_dataframe('../basin_boundary/GRIT_full_catchment_all_EPSG8857_simplify_final_125km2_subset.gpkg')
join = gpd.overlay(gdf_dam, gdf)
join = join.groupby('ohdb_id')['Feature_ID'].count().rename(columns={'Feature_ID':'dam_num'})
join = join.reindex(gdf.ohdb_id.values).fillna(0).reset_index()
join.to_csv('../geography/dam_num.csv', index = False)

### extract average meteorological conditions in the past 7 days preceding Qmax7 and Qmin7

In [ ]:
# func to calculate averages in a parallel manner
def func_meteo(x):
    x['tmp'] = (x.date - x.Qdate).dt.days
    x3 = x.loc[(x.tmp>-3)&(x.tmp<=0),:].drop(columns=['date','Qdate','ohdb_id','tmp']).mean().rename(index=lambda x:x+'_3')
    x7 = x.loc[(x.tmp>-7)&(x.tmp<=0),:].drop(columns=['date','Qdate','ohdb_id','tmp']).mean().rename(index=lambda x:x+'_7')
    x15 = x.loc[(x.tmp>-15)&(x.tmp<=0),:].drop(columns=['date','Qdate','ohdb_id','tmp']).mean().rename(index=lambda x:x+'_15')
    x30 = x.loc[(x.tmp>-30)&(x.tmp<=0),:].drop(columns=['date','Qdate','ohdb_id','tmp']).mean().rename(index=lambda x:x+'_30')
    x = pd.concat([x3,x7,x15,x30])
    return x

df_flood = pd.read_csv('../data/dis_OHDB_Qmin7_Qmax7_1982-2023.csv')
df_flood['Qmax7date'] = pd.to_datetime(df_flood['Qmax7date'])
df_flood['Qmin7date'] = pd.to_datetime(df_flood['Qmin7date'])

for year in range(1982, 2024):
    if os.path.exists(f'../data/Qmax7_meteo_multi_{year}.csv') and os.path.exists(f'../data/Qmin7_meteo_multi_{year}.csv'):
        continue
    fname = f'../ee_era5_land/ERA5_Land_daily_meteorology_for_OHDB_10717_stations_{year}.csv'
    df = pd.read_csv(fname)
    df = df.melt(id_vars = 'ohdb_id')
    df['date'] = df.variable.apply(lambda x:x[:4]+'-'+x[4:6]+'-'+x[6:8])
    df['meteo'] = df.variable.str[9:]
    df = df.drop(columns=['variable'])
    df = df.pivot_table(index = ['ohdb_id','date'], columns = 'meteo', values = 'value').reset_index()
    df['date'] = pd.to_datetime(df.date.values)

    # change meteo unit
    df[['snow_depth_water_equivalent','snowmelt_sum','total_evaporation_sum','total_precipitation_sum']] = df[['snow_depth_water_equivalent','snowmelt_sum','total_evaporation_sum','total_precipitation_sum']] * 1000
    df[['snow_depth_water_equivalent','snowmelt_sum','total_evaporation_sum','total_precipitation_sum']] = df[['snow_depth_water_equivalent','snowmelt_sum','total_evaporation_sum','total_precipitation_sum']].round(3)
    df[['temperature_2m_min','temperature_2m_max']] = df[['temperature_2m_min','temperature_2m_max']] - 273.15
    df[['temperature_2m_min','temperature_2m_max']] = df[['temperature_2m_min','temperature_2m_max']].round(3)
    df['surface_net_solar_radiation_sum'] = df['surface_net_solar_radiation_sum'] * 1e-6
    df['surface_net_solar_radiation_sum'] = df['surface_net_solar_radiation_sum'].round(3)

    # change meteo name
    df = df.rename(columns = {
        'snow_depth_water_equivalent':'swe',
        'snowmelt_sum':'swmelt',
        'total_evaporation_sum':'evap',
        'total_precipitation_sum':'pr',
        'temperature_2m_min':'t2min',
        'temperature_2m_max':'t2max',
        'surface_net_solar_radiation_sum':'srad'
    })

    df_flood0 = df_flood.loc[df_flood.year==year,:]

    df_Qmax7 = df.merge(df_flood0[['ohdb_id','Qmax7date']].rename(columns={'Qmax7date':'Qdate'}), on = 'ohdb_id')
    print(df_Qmax7.shape)
    df_Qmax7 = df_Qmax7.groupby('ohdb_id').p_apply(func_meteo).reset_index().assign(year=year)
    df_Qmax7 = df_Qmax7.merge(df_flood0[['ohdb_id','Qmax7','Qmax7date']], on = 'ohdb_id')
    df_Qmax7.to_csv(f'../data/Qmax7_meteo_multi_{year}.csv', index = False)

    df_Qmin7 = df.merge(df_flood0[['ohdb_id','Qmin7date']].rename(columns={'Qmin7date':'Qdate'}), on = 'ohdb_id')
    print(df_Qmin7.shape)
    df_Qmin7 = df_Qmin7.groupby('ohdb_id').p_apply(func_meteo).reset_index().assign(year=year)
    df_Qmin7 = df_Qmin7.merge(df_flood0[['ohdb_id','Qmin7','Qmin7date']], on = 'ohdb_id')
    df_Qmin7.to_csv(f'../data/Qmin7_meteo_multi_{year}.csv', index = False)

### merge basin attributes

In [ ]:
fnames = glob.glob('../geography/*csv')
df_all = []
for fname in fnames:
    df = pd.read_csv(fname)
    if df.shape[0] == 1:
        df = df.T
    if df.shape[0] == 10717:
        df = df.set_index('ohdb_id')
    name = os.path.basename(fname).split('_')[0]
    if '0-5cm' in fname:
        name = name + '_layer1'
    elif '5-15cm' in fname:
        name = name + '_layer2'
    elif '15-30cm' in fname:
        name = name + '_layer3'
    elif '30-60cm' in fname:
        name = name + '_layer4'
    elif '60-100cm' in fname:
        name = name + '_layer5'
    elif '100-200cm' in fname:
        name = name + '_layer6'
    if df.shape[1] == 1:
        df.columns = [name]
    df_all.append(df)
df_all = pd.concat(df_all, axis = 1)
df_all = df_all.loc[df_all.index.str.contains('OHDB'),:].reset_index().rename(columns={'index':'ohdb_id'})

# merge metadata
df_meta = pd.read_csv('../OHDB_metadata_subset.csv')
df_all = df_all.merge(df_meta, on = 'ohdb_id')

df_all.to_csv('../basin_attributes.csv', index = False)


### use GDAT reservoir area / catchment area to indicate the impact of reservoir regulation

In [89]:
gdf_res = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_catchments.shp').to_crs('epsg:8857')
gdf_dam = read_dataframe('../../data/geography/GDAT_data_v1/data/GDAT_v1_dams.shp').to_crs('epsg:8857')
gdf_basin = read_dataframe('../basin_boundary/GRIT_full_catchment_all_EPSG8857_simplify_final_125km2_subset.gpkg')

In [90]:
inter = gpd.sjoin(gdf_basin, gdf_dam)
gdf_res['darea'] = gdf_res.area / 1000000
inter = inter.merge(gdf_res[['Feature_ID','Dam_Name','darea']], on = ['Feature_ID','Dam_Name'])
inter.loc[inter.Year_Fin=='BLANK','Year_Fin'] = None
inter['Year_Fin'] = pd.to_numeric(inter['Year_Fin'])

inter.loc[inter.Year_Const.isna(),'Year_Const'] = np.nan
inter.loc[~inter.Year_Const.isna(),'Year_Const'] = inter.loc[~inter.Year_Const.isna(),'Year_Const'].str[:4].astype(int)
inter['year'] = inter[['Year_Fin','Year_Const']].min(axis=1)
inter = inter.groupby(['ohdb_id','gritDarea']).apply(lambda x:pd.Series([
    x.darea.sum(),
    x.year.mean(),
    x.Main_P_Map.mode().values[0] if len(x.Main_P_Map.mode().values)>=1 else 'Hydroelectricity'
], index = ['res_darea_normalize','Year_ave','Main_Purpose_mode'])).reset_index()
inter['Year_ave'] = inter['Year_ave'].fillna(2000)
inter['Main_Purpose_mode'] = inter['Main_Purpose_mode'].fillna('Hydroelectricity')
inter['res_darea_normalize'] = inter.res_darea_normalize / inter.gritDarea
inter.to_csv('../data/dam_impact.csv', index = False)
print(inter.head())

          ohdb_id      gritDarea  res_darea_normalize  Year_ave  \
0  OHDB_001000002  235955.440350             0.000016    2000.0   
1  OHDB_001000014   12442.131563             0.042140    2000.0   
2  OHDB_001000015  125057.617313             0.039645    2000.0   
3  OHDB_001000017    3013.998413             0.013224    2003.0   
4  OHDB_001000018   19484.903813             0.002046    2003.0   

  Main_Purpose_mode  
0             Other  
1  Hydroelectricity  
2             Other  
3  Hydroelectricity  
4  Hydroelectricity  


In [86]:
inter.Year_ave.isna().sum()

589

In [51]:
gdf_dam.Year_Const.values[0] == None

True